In [3]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 19:11:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
!aws --profile=devops s3 cp s3://nyc-tlc/csv_backup/fhvhv_tripdata_2021-01.csv .

download: s3://nyc-tlc/csv_backup/fhvhv_tripdata_2021-01.csv to ./fhvhv_tripdata_2021-01.csv


In [3]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')


[Stage 0:>                                                          (0 + 1) / 1]



In [4]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [5]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [6]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [7]:
df_pandas = pd.read_csv('head.csv')

In [8]:
!head head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag

HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,

HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,

HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,

HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,

HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,

HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,

HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,

HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,

HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,



In [9]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [12]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [13]:
from pyspark.sql import types

In [14]:
schema = types.StructType([
  types.StructField('hvfhs_license_num', types.StringType(), True), 
  types.StructField('dispatching_base_num', types.StringType(), True), 
  types.StructField('pickup_datetime', types.TimestampType(), True), 
  types.StructField('dropoff_datetime', types.TimestampType(), True), 
  types.StructField('PULocationID', types.IntegerType(), True), 
  types.StructField('DOLocationID', types.IntegerType(), True), 
  types.StructField('SR_Flag', types.StringType(), True)
])

In [15]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [16]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [17]:
df = df.repartition(24)

In [18]:
df.write.parquet('fhvhv/2021/01/')

22/10/05 19:20:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/10/05 19:21:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/10/05 19:21:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [19]:
df = spark.read.parquet('fhvhv/2021/01/')

In [20]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [21]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == "HV0003") \
  .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 08:28:06|2021-01-01 08:47:57|         159|          51|
|2021-01-02 19:45:07|2021-01-02 19:53:50|         231|         261|
|2021-01-04 16:43:20|2021-01-04 16:50:13|          75|         151|
|2021-01-01 06:25:50|2021-01-01 06:34:11|          35|         177|
|2021-01-03 04:12:08|2021-01-03 04:29:53|          17|          61|
|2021-01-04 17:14:48|2021-01-04 17:30:28|          33|         255|
|2021-01-01 07:37:26|2021-01-01 07:41:28|         169|         169|
|2021-01-03 10:56:01|2021-01-03 11:01:16|         188|          89|
|2021-01-01 07:36:31|2021-01-01 07:47:44|         168|          41|
|2021-01-03 10:31:41|2021-01-03 10:47:51|          65|         137|
|2021-01-03 13:12:32|2021-01-03 13:25:28|         188|          72|
|2021-01-04 17:07:35|2021-01-04 17:25:44|       

In [22]:
from pyspark.sql import functions as F

In [26]:
df.withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
  .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
  .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
  .show()          

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-01|  2021-01-01|         159|          51|
| 2021-01-05|  2021-01-05|          35|         188|
| 2021-01-05|  2021-01-05|         169|          18|
| 2021-01-02|  2021-01-02|          75|          47|
| 2021-01-02|  2021-01-02|         231|         261|
| 2021-01-04|  2021-01-04|          75|         151|
| 2021-01-01|  2021-01-01|          35|         177|
| 2021-01-02|  2021-01-02|          21|          11|
| 2021-01-04|  2021-01-04|         243|         166|
| 2021-01-03|  2021-01-03|          17|          61|
| 2021-01-04|  2021-01-04|         158|         231|
| 2021-01-04|  2021-01-04|          33|         255|
| 2021-01-01|  2021-01-01|         169|         169|
| 2021-01-03|  2021-01-03|         188|          89|
| 2021-01-01|  2021-01-01|         168|          41|
| 2021-01-03|  2021-01-03|          65|       